In [6]:
import numpy as np
import torch
import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Load breast cancer dataset
data = datasets.load_breast_cancer()
X, y = data.data, data.target
X = X.astype(np.float32)

# Preprocess the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Define the model and guide
def model(X, y):
    n_features = X.shape[1]
    w = pyro.sample("w", dist.Normal(torch.zeros(n_features), torch.ones(n_features)).to_event(1))
    b = pyro.sample("b", dist.Normal(0., 1.))
    logits = torch.matmul(X, w) + b
    with pyro.plate("data", len(X)):
        pyro.sample("obs", dist.Bernoulli(logits=logits), obs=y)

def guide(X, y):
    n_features = X.shape[1]
    w_loc = pyro.param("w_loc", torch.zeros(n_features))
    w_scale = pyro.param("w_scale", torch.ones(n_features), constraint=dist.constraints.positive)
    b_loc = pyro.param("b_loc", torch.tensor(0.))
    b_scale = pyro.param("b_scale", torch.tensor(1.), constraint=dist.constraints.positive)

    w = pyro.sample("w", dist.Normal(w_loc, w_scale).to_event(1))
    b = pyro.sample("b", dist.Normal(b_loc, b_scale))


# Perform Stochastic Variational Inference
pyro.clear_param_store()
optimizer = Adam({"lr": 0.01})
svi = SVI(model, guide, optimizer, loss=Trace_ELBO())

num_iterations = 1000
for i in range(num_iterations):
    svi.step(X_train_tensor, y_train_tensor)

# Make predictions on the test set
w_posterior = pyro.param("w_loc")
b_posterior = pyro.param("b_loc")
test_logits = torch.matmul(X_test_tensor, w_posterior) + b_posterior
y_test_preds = torch.round(torch.sigmoid(test_logits)).detach().numpy()

# Calculate accuracy
accuracy = accuracy_score(y_test, y_test_preds)
print("Accuracy with Pyro Bayesian Logistic Regression: {:.2f}%".format(accuracy * 100))


Accuracy with Pyro Bayesian Logistic Regression: 98.83%


In [10]:
import torch
import pyro
import pyro.distributions as dist
from pyro.contrib.cevae import CEVAE

def generate_data(num_data, feature_dim):
    z = dist.Bernoulli(0.5).sample([num_data])
    x = dist.Normal(z, 5 * z + 3 * (1 - z)).sample([feature_dim]).t()
    t = dist.Bernoulli(0.75 * z + 0.25 * (1 - z)).sample()
    y = dist.Bernoulli(logits=3 * (z + 2 * (2 * t - 2))).sample()
    t0_t1 = torch.tensor([[0.0], [1.0]])
    y_t0, y_t1 = dist.Bernoulli(logits=3 * (z + 2 * (2 * t0_t1 - 2))).mean
    true_ite = y_t1 - y_t0
    return x, t, y, true_ite

def main(args):
    if args.cuda:
        torch.set_default_tensor_type("torch.cuda.FloatTensor")

    pyro.set_rng_seed(args.seed)
    x_train, t_train, y_train, true_ite = generate_data(args.num_data, args.feature_dim)

    pyro.set_rng_seed(args.seed)
    pyro.clear_param_store()
    cevae = CEVAE(
        feature_dim=args.feature_dim,
        latent_dim=args.latent_dim,
        hidden_dim=args.hidden_dim,
        num_layers=args.num_layers,
        num_samples=10,
    )
    cevae.fit(
        x_train,
        t_train,
        y_train,
        num_epochs=args.num_epochs,
        batch_size=args.batch_size,
        learning_rate=args.learning_rate,
        learning_rate_decay=args.learning_rate_decay,
        weight_decay=args.weight_decay,
    )

    true_ate = true_ite.mean()
    print("true ATE = {:0.3g}".format(true_ate.item()))
    naive_ate = y_train[t_train == 1].mean() - y_train[t_train == 0].mean()
    print("naive ATE = {:0.3g}".format(naive_ate))
    if args.jit:
        cevae = cevae.to_script_module()
    est_ite = cevae.ite(x_train)
    est_ate = est_ite.mean()
    print("estimated ATE = {:0.3g}".format(est_ate.item()))


class Args:
    def __init__(
        self,
        num_data=1000,
        feature_dim=5,
        latent_dim=20,
        hidden_dim=200,
        num_layers=3,
        num_epochs=50,
        batch_size=100,
        learning_rate=1e-3,
        learning_rate_decay=0.1,
        weight_decay=1e-4,
        seed=1234567890,
        jit=False,
        cuda=False,
    ):
        self.num_data = num_data
        self.feature_dim = feature_dim
        self.latent_dim = latent_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.num_epochs = num_epochs
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.learning_rate_decay = learning_rate_decay
        self.weight_decay = weight_decay
        self.seed = seed
        self.jit = jit
        self.cuda = cuda

args = Args()
main(args)


INFO 	 Training with 10 minibatches per epoch
DEBUG 	 step     0 loss = 12.6351
DEBUG 	 step   100 loss = 8.86371
DEBUG 	 step   200 loss = 9.06821
DEBUG 	 step   300 loss = 8.93611
DEBUG 	 step   400 loss = 9.163
INFO 	 Evaluating 1 minibatches


true ATE = 0.728
naive ATE = 0.812


DEBUG 	 batch ate = 0.792038


estimated ATE = 0.792
